# Stockformer Demo

In [ ]:
import sys

# if not 'Informer2020' in sys.path:
#     sys.path += ['Informer2020']

## Open log_dir

In [ ]:
from utils.tools import dotdict
import torch
import numpy as np
import pandas as pd
import os
from pprint import pprint
import matplotlib.pyplot as plt
from utils.ipynb_helpers import (
    args_from_setting,
    setting_from_args,
    handle_gpu,
    read_data,
)
import yaml
from utils.stock_metrics import (
    apply_threshold_metric,
    PctProfitDirection,
    PctProfitTanh,
    LogPctProfitDirection,
    LogPctProfitTanh,
    pct_direction,
)
log_dir = "lightning_logs/stockformer_custom_ftMS_sl16_ll0_pl1_ei9_diNone_co1_iNone_dm512_nh512_el12_dlNone_df2048_atfull_fc5_ebNone_dtFalse_mxFalse_full_1h_0/version_1"
# log_dir = "lightning_logs/stockformer_custom_ftMS_sl16_ll0_pl1_ei9_diNone_co1_iNone_dm256_nh256_el12_dlNone_df2048_atfull_fc5_ebNone_dtFalse_mxFalse_full_1h_0/version_11"
# log_dir = "lightning_logs/stockformer_custom_ftMS_sl16_ll0_pl1_ei9_diNone_co1_iNone_dm128_nh128_el12_dlNone_df2048_atfull_fc5_ebNone_dtFalse_mxFalse_full_1h_0/version_4"
# log_dir = "lightning_logs/stockformer_custom_ftMS_sl16_ll0_pl1_ei9_diNone_co1_iNone_dm1028_nh512_el12_dlNone_df2048_atfull_fc5_ebNone_dtFalse_mxFalse_full_1h_0/version_0"
# log_dir = "lightning_logs/stockformer_custom_ftMS_sl16_ll0_pl1_ei9_diNone_co1_iNone_dm512_nh512_el12_dlNone_df2048_atprob_fc5_ebNone_dtFalse_mxFalse_full_1h_0/version_25"
# log_dir = "lightning_logs/stockformer_custom_ftMS_sl16_ll0_pl1_ei9_diNone_co1_iNone_dm256_nh256_el12_dlNone_df2048_atprob_fc5_ebNone_dtFalse_mxFalse_full_1h_0/version_0"
log_dir = "lightning_logs/stockformer_custom_ftMS_sl16_ll0_pl1_ei9_diNone_co1_iNone_dm512_nh512_el12_dlNone_df2048_atfull_fc5_ebNone_dtFalse_mxFalse_full_1h_0/version_9"
with open(os.path.join(log_dir, "hparams.yaml"), "r") as file:
    args = dotdict(yaml.load(file, Loader=yaml.FullLoader))

args

In [ ]:
df = read_data(os.path.join(args.root_path,args.data_path))
df.head()

## Visualization

In [ ]:
# When we finished exp.train(setting) and exp.test(setting), we will get a trained model and the results of test experiment
# The results of test experiment will be saved in ./results/{setting}/pred.npy (prediction of test dataset) and ./results/{setting}/true.npy (groundtruth of test dataset)

tp_dict = {}
for flag in ["train", "val", "test"]:
    device = 0
    while True: # Device Loop
        preds_path = os.path.join(log_dir, f"results/pred_{flag}_{device}.npy")
        trues_path = os.path.join(log_dir, f"results/true_{flag}_{device}.npy")
        dates_path = os.path.join(log_dir, f"results/date_{flag}_{device}.npy")
        if (
            os.path.exists(preds_path)
            and os.path.exists(trues_path)
            and os.path.exists(dates_path)
        ):
            dp = [np.load(trues_path)[:,0,0], np.load(preds_path)[:,0,0], np.load(dates_path)[:,0]]
            tp_dict[flag] = dp if flag not in tp_dict else [np.append(tpdfi, dpi,axis=0) for tpdfi, dpi in zip(tp_dict[flag], dp)]
            s = np.argsort(tp_dict[flag][2], axis=None)
            tp_dict[flag] = list(map(lambda x: x[s], tp_dict[flag]))

            tp_dict[flag][2] = pd.DatetimeIndex(tp_dict[flag][2], tz="UTC")

            # Override trues with df target data
            if not("mse" in args.loss and not args.inverse_output):
                print("OVERRIDING trues with df target ")
                df_flag = df.loc[tp_dict[flag][2]]
                t = args.target.split("_")
                df_target = df_flag[t[0]][t[1]].to_numpy()
                tp_dict[flag][0] = df_target

        else:
            # Done searching for devices
            break
        device+=1


print("Open true/pred data for:", list(tp_dict.keys()))

# [samples, pred_len, dimensions]
print(
    tp_dict["train"][0].shape, tp_dict["val"][0].shape, tp_dict["test"][0].shape, "\n\n"
)

for flag in tp_dict:
    trues, preds, dates = tp_dict[flag]
    print(
        f"{flag}\ttrues.shape: {trues.shape}, preds.shape: {preds.shape}, dates.shape: {dates.shape}"
    )

    MSE = np.square(np.subtract(trues, preds)).mean()
    RMSE = np.sqrt(MSE)
    print("against preds", MSE, RMSE)

    MSE = np.square(np.subtract(trues, np.zeros(preds.shape))).mean()
    RMSE = np.sqrt(MSE)
    print("against 0s", MSE, RMSE)

In [ ]:
# draw OT prediction
for flag in tp_dict:
    true, pred, date = tp_dict[flag]

    if "stock" in args.loss:
        true = true / np.linalg.norm(true)
        pred = pred / np.linalg.norm(pred)

    # pred = pred.copy()*10 #* (np.random.random(pred.shape)-.5)*100
    plt.figure(num=flag, figsize=(16, 4))
    plt.title(flag)
    plt.plot(date, true, label="GroundTruth", linestyle="", marker=".", markersize=4)
    plt.plot(date, pred, label="Prediction", linestyle="", marker=".", markersize=4)
    plt.plot(date, np.zeros(date.shape), color="red")
    # plt.scatter(range(trues.shape[0]), trues[:,0,0], marker='v', color='r', label='GroundTruth')
    # plt.scatter(range(trues.shape[0]), preds[:,0,0], marker='^', color='m', label='Prediction')

    plt.legend()
    plt.show()

    plt.figure(num=flag, figsize=(16, 4))
    plt.title("Diff histogram")
    # plt.hist(np.abs(true), bins=len(true)//6, label='Diff 0', alpha=0.5)
    # plt.hist(np.abs(true - pred), bins=len(true)//6, label='Diff Pred', alpha=0.5)
    plt.hist(
        [np.abs(true), np.abs(true - pred)], bins=60, label=["Diff 0", "Diff Pred"]
    )
    plt.xlabel("Diff Value")
    plt.ylabel("Count")
    plt.legend()
    plt.show()

    # df = pd.concat([pd.DataFrame(a, columns=[f"{i}"]) for i, a in enumerate([np.abs(true - pred), np.abs(true)])], axis=1)

    # # plot the data
    # df.plot.hist(stacked=True, bins=len(true), density=True, figsize=(10, 6), grid=True)

## Basic back-test based on buying in predicted direction if prediction is above a threshold

In [ ]:
max_tracker = (0, 0)

# Tracks results
tracker = {}

df = read_data(os.path.join(args.root_path, args.data_path))

# Get the percentile to check thresh until
percentile = [50, 0.0]
for flag in ["train"]:  # tp_dict:
    _, preds, _ = tp_dict[flag]
    percentile[1] += np.percentile(
        np.abs(preds), percentile[0]
    ) 
    print(np.median(np.abs(preds)))
# percentile[1] /= len(tp_dict)
print(f"{percentile[0]}'th percentile: {percentile[1]}")

ticker, field = args.target.split("_")
assert field == "pctchange" or field == "logpctchange"

for thresh in np.linspace(0, percentile[1], 501):
    # print("thresh:", thresh)
    tracker[thresh] = {}
    track = {}
    for flag in tp_dict:
        true, pred, date = tp_dict[flag]

        # Filter by thresh. Note in log scale
        pred_f, true_f = apply_threshold_metric(pred, true, thresh)
        df_f = df.loc[date[np.abs(pred) >= thresh]]

        # Percent direction correct, ie up or down
        pct_dir_correct = pct_direction(pred_f, true_f)

        # Percent profit all in
        pct_profit_dir = LogPctProfitDirection.metric(pred_f, true_f, short_filter=None)
        pct_profit_dir_nshort = LogPctProfitDirection.metric(pred_f, true_f, short_filter="ns")
        pct_profit_dir_oshort = LogPctProfitDirection.metric(pred_f, true_f, short_filter="os")

        # Percent profit with tanh partial purchase
        pct_profit_tanh = LogPctProfitTanh.metric(pred_f, true_f, short_filter=None)
        pct_profit_tanh_nshort = LogPctProfitTanh.metric(pred_f, true_f, short_filter="ns")
        pct_profit_tanh_oshort = LogPctProfitTanh.metric(pred_f, true_f, short_filter="os")

        # Optimal percent profit
        pct_profit_dir_opt = LogPctProfitDirection.metric(true_f, true_f)

        # Tune threshhold based off of train's metric we care about
        tune_metric = pct_profit_tanh if args.loss == "stock_tanh" else pct_profit_dir
        if tune_metric > max_tracker[0] and flag == "train":
            max_tracker = (tune_metric, thresh)

        # Save
        tracker[thresh][flag] = {
            "pct_profit_dir": pct_profit_dir,
            "pct_profit_dir_nshort": pct_profit_dir_nshort,
            "pct_profit_dir_oshort": pct_profit_dir_oshort,
            "pct_profit_tanh": pct_profit_tanh,
            "pct_profit_tanh_nshort": pct_profit_tanh_nshort,
            "pct_profit_tanh_oshort": pct_profit_tanh_oshort,
            "pct_excluded": (len(pred) - len(pred_f)) / len(pred), 
            "pct_excluded_nshort": (len(pred) - len(pred_f[pred_f > 0])) / len(pred),
            "pct_excluded_oshort": (len(pred) - len(pred_f[pred_f < 0])) / len(pred),
            "pct_dir_correct": pct_dir_correct,
            "pct_profit_dir_opt": pct_profit_dir_opt,
        }


best_thresh = max_tracker[1]
print("best thresh:", best_thresh)
for data_group in tracker[best_thresh]:
    print(data_group, end="\t") 
    pprint(tracker[best_thresh][data_group], indent=3)


In [ ]:
fig, axs = plt.subplots(4,1, sharex=True, figsize=(16, 8))

for flag in ["train", "val", "test"]: #tp_dict:
    true, pred, date = tp_dict[flag]

    # if flag == "train":
    #     true = true[:1000]
    #     pred = pred[:1000]
    #     date = date[:1000]


    # Filter by best_thresh. Note in log scale
    pred_f, true_f = apply_threshold_metric(pred, true, best_thresh)
    date_f = date[np.abs(pred) >= best_thresh]
    df_f = df.loc[date[np.abs(pred) >= best_thresh]]

    if "lpp" in args.loss:
        metric = LogPctProfitDirection
        metric_name = "pct_profit_dir"
    elif "tanh" in args.loss:
        metric = LogPctProfitTanh
        metric_name = "pct_profit_tanh"
    elif "mse" in args.loss:
        metric = LogPctProfitDirection
        metric_name = "pct_profit_dir"
    elif "mae" in args.loss:
        metric = LogPctProfitDirection
        metric_name = "pct_profit_dir"

    axs[0].plot(date_f, metric.accumulate(pred_f, true_f, short_filter=None), label=flag)
    axs[0].set_ylabel(metric_name)
    axs[0].set_title(metric_name)
    axs[0].grid(axis = 'y')

    axs[1].plot(date_f[pred_f > 0], metric.accumulate(pred_f, true_f, short_filter="ns"))#, label=flag)
    axs[1].set_ylabel(f"{metric_name}_nshort")
    axs[1].set_title(f"{metric_name}_nshort")
    axs[1].grid(axis = 'y')

    axs[2].plot(date_f[pred_f < 0], metric.accumulate(pred_f, true_f, short_filter="os"))#, label=flag)
    axs[2].set_ylabel(f"{metric_name}_oshort")
    axs[2].set_title(f"{metric_name}_oshort")
    axs[2].grid(axis = 'y')

    axs[3].plot(date_f, np.exp(np.cumsum(true_f)), label="Market")
    # axs[3].set_ylabel("Market")
    axs[3].set_title("Market")
    axs[3].grid(axis = 'y')

fig.legend()
fig.suptitle("Cumulative metrics overtime")

fig.show()

## Attention Visualization

In [ ]:
# args.output_attention = True

# exp = Exp(args)

# model = exp.model

# path = os.path.join(args.checkpoints, setting, "checkpoint.pth")

# print(model.load_state_dict(torch.load(path)))

# df = pd.read_csv(os.path.join(args.root_path, args.data_path))
# df[args.cols].head()

In [ ]:
# from data_provider.data_loader import Dataset_Custom
# from torch.utils.data import DataLoader

# Data = Dataset_Custom
# timeenc = 0 if args.embed != "timeF" else 1
# flag = "test"
# shuffle_flag = False
# drop_last = True
# batch_size = 1
# data_set = Data(args, flag=flag)

# data_loader = DataLoader(
#     data_set,
#     batch_size=batch_size,
#     shuffle=shuffle_flag,
#     num_workers=args.num_workers,
#     drop_last=drop_last,
# )


# idx = 0
# for i, (batch_x, batch_y, batch_x_mark, batch_y_mark, ds_index) in enumerate(
#     data_loader
# ):
#     if i != idx:
#         continue
#     batch_x = batch_x.float().to(exp.device)
#     batch_y = batch_y.float()

#     batch_x_mark = batch_x_mark.float().to(exp.device)
#     batch_y_mark = batch_y_mark.float().to(exp.device)

#     dec_inp = torch.zeros_like(batch_y[:, -args.pred_len :, :]).float()
#     dec_inp = (
#         torch.cat([batch_y[:, : args.label_len, :], dec_inp], dim=1)
#         .float()
#         .to(exp.device)
#     )

#     outputs, attn = model(batch_x, batch_x_mark, dec_inp, batch_y_mark)


# print(attn[0].shape, attn[1].shape)  # , attn[2].shape


# layers = [0, 1]
# distil = "Distil" if args.distil else "NoDistil"
# for layer in layers:
#     print("\n\n==========================")
#     print("Showing attention layer", layer)
#     print("==========================\n\n")
#     for h in range(0, args.n_heads):
#         plt.figure(figsize=[10, 8])
#         plt.title(f"Informer, {distil}, attn:{args.attn} layer:{layer} head:{h}")
#         A = attn[layer][0, h].detach().cpu().numpy()
#         ax = sns.heatmap(A, vmin=0, vmax=A.max() + 0.01)
#         plt.show()

In [ ]:
import pytorch_lightning as pl
from exp.exp_timeseries import ExpTimeseries
from data_provider.data_module import CustomDataModule

trainer = pl.Trainer(accelerator="gpu",devices=1)#, log_dir=os.path.abspath(log_dir))

exp = ExpTimeseries.load_from_checkpoint(
    os.path.join(log_dir, "checkpoints/checkpoint.ckpt"), config=args
)
data_module = CustomDataModule(args, 0)

# Test Model
# t = trainer.test(exp, data_module)

# # Predict and Save Results
results = trainer.predict(exp, data_module)

In [ ]:
results